# Download and Save DAIC-WOZ Files (zip) to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p /content/daic_tmp
%cd /content/daic_tmp

In [ ]:
%%shell
base_url="https://dcapswoz.ict.usc.edu/wwwdaicwoz/"

# Example: Download zip files for IDs 300 to 492
for i in {300..492}; do
  filename="${i}_P.zip"
  echo "Downloading $filename from $base_url/$filename"

  # --continue: If the download is interrupted, attempt to resume
  wget --continue "$base_url/$filename"
done

In [ ]:
# Create a folder in Drive for saving (if it doesn't already exist)
!mkdir -p /content/drive/MyDrive/DAIC-WOZ

# Copy files from the temporary directory to Drive
!cp /content/daic_tmp/*.zip /content/drive/MyDrive/DAIC-WOZ/

# Unzip and Save DAIC-WOZ Files from Google Drive in Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Folder in Google Drive where the downloaded zip files are located
# For example: /content/drive/MyDrive/DAIC-WOZ/zips
ZIPS_DIR = '/content/drive/MyDrive/DAIC-WOZ'

# Directory for extracting files (temporary folder)
EXTRACT_DIR = '/content/daic_extracted'

# Directory to save the cleaned transcripts after preprocessing
CLEANED_DIR = '/content/drive/MyDrive/DAIC-WOZ/cleaned_transcripts'

In [ ]:
import os
import glob

os.makedirs(EXTRACT_DIR, exist_ok=True)

# Example for extracting zip files with IDs from 300 to 492
for pid in range(300, 493):
    zip_filename = f"{pid}_P.zip"
    zip_path = os.path.join(ZIPS_DIR, zip_filename)

    # Check if the zip file exists
    if not os.path.exists(zip_path):
        print(f"Not found: {zip_path}")
        continue

    # Destination folder for extraction (e.g., /content/daic_extracted/300_P/)
    out_dir = os.path.join(EXTRACT_DIR, f"{pid}_P")
    os.makedirs(out_dir, exist_ok=True)

    # Extraction in the Colab environment
    !unzip -q -o "{zip_path}" -d "{out_dir}"
    print(f"Extracted {zip_filename} to {out_dir}")

In [ ]:
# Destination folder on Drive
EXTRACT_DIR = '/content/drive/MyDrive/DAIC-WOZ/extracted'
!mkdir -p "{EXTRACT_DIR}"

# Copy from temporary folder to Drive
!cp -r /content/daic_extracted/* "{EXTRACT_DIR}/"

# Vocal Modality Prep (Extracting and Merging Patient Voice Segments)

In [ ]:
!pip install pydub
import os
import pandas as pd
from pydub import AudioSegment

TRANSCRIPT_BASE_DIR = "/content/drive/MyDrive/DAIC-WOZ/extracted"
OUTPUT_DIR = "/content/drive/MyDrive/DAIC-WOZ/processed_patient_segments"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_transcript_custom(path):
    """
    The first line is assumed to be the header ('start_time stop_time speaker value').
    The subsequent lines are parsed manually:
      - The first 3 tokens are start_time, stop_time, speaker
      - Combine the 4th token onwards into value
    """
    rows = []
    with open(path, 'r', encoding='utf-8', errors='replace') as f:
        # Skip the first line (header)
        header_line = next(f, None)

        for line in f:
            line = line.strip()
            if not line:
                continue
            # Split into up to 4 parts by whitespace
            # parts[0] -> start_time
            # parts[1] -> stop_time
            # parts[2] -> speaker
            # parts[3] -> value (full utterance)
            parts = line.split(maxsplit=3)
            if len(parts) < 4:
                # If fewer than 4 parts are obtained, skip or adjust
                continue

            start_str, stop_str, spk, val = parts
            # Convert to numeric values
            start_time = float(start_str)
            stop_time  = float(stop_str)

            rows.append({
                'start_time': start_time,
                'stop_time':  stop_time,
                'speaker':    spk,
                'value':      val
            })

    df = pd.DataFrame(rows,
        columns=['start_time','stop_time','speaker','value']
    )
    return df


def extract_patient_segments(audio_id, n_merge=5):
    """
    1) Read /extracted/{audio_id}_P/{audio_id}_TRANSCRIPT.csv with the custom parser above.
    2) Extract only the segments where speaker == "Participant".
    3) Cut out segments from the actual .wav using pydub according to start_time and stop_time.
    4) Concatenate every n_merge segments and write them to OUTPUT_DIR.
    5) Return a list of output file paths.
    """
    subfolder = f"{audio_id}_P"
    transcript_file = f"{audio_id}_TRANSCRIPT.csv"
    audio_file      = f"{audio_id}_AUDIO.wav"

    transcript_path = os.path.join(TRANSCRIPT_BASE_DIR, subfolder, transcript_file)
    audio_path      = os.path.join(TRANSCRIPT_BASE_DIR, subfolder, audio_file)

    # Read the DataFrame using the custom parser
    df = load_transcript_custom(transcript_path)

    # Only Participant rows
    df_patient = df[df['speaker'] == 'Participant'].copy()

    # Load the entire audio using pydub
    audio_full = AudioSegment.from_wav(audio_path)

    segments = []
    for _, row in df_patient.iterrows():
        start_ms = row['start_time'] * 1000.0
        end_ms   = row['stop_time']  * 1000.0
        # Clip/cut out the segment
        seg = audio_full[start_ms:end_ms]
        segments.append(seg)

    # Concatenate every n_merge segments
    out_files = []
    merged_count = 0
    for i in range(0, len(segments), n_merge):
        chunk_segs = segments[i:i+n_merge]
        if not chunk_segs:
            continue
        merged = sum(chunk_segs)

        out_filename = f"{audio_id}_merged_{merged_count}.wav"
        out_path = os.path.join(OUTPUT_DIR, out_filename)
        merged.export(out_path, format="wav")
        out_files.append(out_path)

        merged_count += 1

    return out_files

# --- Example usage ---
files_300 = extract_patient_segments(audio_id=300, n_merge=5)
print(files_300)

# Visual Modality Prep (Merging CLNF Data)

In [ ]:
import os
import pandas as pd
import numpy as np
from typing import List

TRANSCRIPT_BASE_DIR = "/content/drive/MyDrive/DAIC-WOZ/extracted"
VISUAL_OUTPUT_DIR = "/content/drive/MyDrive/DAIC-WOZ/processed_visual_segments"
os.makedirs(VISUAL_OUTPUT_DIR, exist_ok=True)

def load_transcript_custom(path):
    """
    The first line is assumed to be a header.
    Read the subsequent lines, separating them into four parts:
      - start_time, stop_time, speaker, value
    """
    rows = []
    with open(path, 'r', encoding='utf-8', errors='replace') as f:
        # Skip the first line (header)
        header_line = next(f, None)
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(maxsplit=3)
            if len(parts) < 4:
                continue
            start_str, stop_str, spk, val = parts
            start_time = float(start_str)
            stop_time  = float(stop_str)

            rows.append({
                'start_time': start_time,
                'stop_time':  stop_time,
                'speaker':    spk,
                'value':      val
            })

    df = pd.DataFrame(rows, columns=['start_time','stop_time','speaker','value'])
    return df

def load_clnf_data(audio_id: int) -> pd.DataFrame:
    """
    Perform an outer join merge of CLNF files on the "timestamp" column.
    - Do not read the hog file.
    - If there are duplicate columns, use suffixes=("", "_dup") to avoid collisions.
    - Strip leading/trailing whitespace from column names.
    """
    subfolder = f"{audio_id}_P"
    folder_path = os.path.join(TRANSCRIPT_BASE_DIR, subfolder)

    clnf_filenames = [
        f"{audio_id}_CLNF_pose.txt",
        f"{audio_id}_CLNF_hog.txt",  # skip hog
        f"{audio_id}_CLNF_gaze.txt",
        f"{audio_id}_CLNF_features3D.txt",
        f"{audio_id}_CLNF_features.txt",
        f"{audio_id}_CLNF_AUs.txt",
    ]

    df_merged = None
    for fn in clnf_filenames:
        fp = os.path.join(folder_path, fn)
        if not os.path.exists(fp):
            print(f"[WARN] Not found: {fp}, skip.")
            continue

        # Skip the hog file
        if "hog" in fn.lower():
            print(f"[INFO] Skipping HOG file: {fn}")
            continue

        try:
            # First try reading using comma as delimiter
            # (If it's actually whitespace-delimited, use delim_whitespace=True)
            with open(fp, "r", encoding="utf-8", errors="replace") as f:
                df_temp = pd.read_csv(f, sep=",", header=0)
        except UnicodeDecodeError:
            print(f"[ERROR] Could not decode {fn} as UTF-8. Skipping...")
            continue

        # Strip leading/trailing whitespace from column names
        df_temp.columns = [col.strip() for col in df_temp.columns]

        # If timestamp column doesn't exist → create one from frames (assuming fps=30)
        if "timestamp" not in df_temp.columns and "frame" in df_temp.columns:
            df_temp["timestamp"] = df_temp["frame"] / 30.0

        # Merge
        if df_merged is None:
            df_merged = df_temp
            df_merged.columns = [col.strip() for col in df_merged.columns]
        else:
            df_merged.columns = [col.strip() for col in df_merged.columns]
            df_merged = pd.merge(df_merged, df_temp,
                                 on="timestamp", how="outer",
                                 suffixes=("", "_dup"))

    if df_merged is None:
        return pd.DataFrame()

    # Sort by timestamp at the end
    df_merged = df_merged.sort_values("timestamp").reset_index(drop=True)
    return df_merged

def extract_visual_segments(audio_id, n_merge=5):
    """
    1. Read the transcript.
    2. Filter to only the rows with speaker == 'Participant'.
    3. Load CLNF data.
    4. For each Participant utterance, find data rows whose timestamps
       lie between start_time and stop_time.
    5. Concatenate every n_merge segments and output them as CSV.
    6. Return the list of output file paths.
    """
    subfolder = f"{audio_id}_P"
    transcript_file = f"{audio_id}_TRANSCRIPT.csv"
    transcript_path = os.path.join(TRANSCRIPT_BASE_DIR, subfolder, transcript_file)

    df_trans = load_transcript_custom(transcript_path)
    df_patient = df_trans[df_trans['speaker'] == 'Participant'].copy()

    df_clnf = load_clnf_data(audio_id)
    if df_clnf.empty:
        print(f"[WARN] No CLNF data loaded for {audio_id}")
        return []

    segments = []
    for _, row in df_patient.iterrows():
        start_sec = row['start_time']
        stop_sec  = row['stop_time']
        mask = (df_clnf["timestamp"] >= start_sec) & (df_clnf["timestamp"] <= stop_sec)
        seg_df = df_clnf[mask].copy()
        segments.append(seg_df)

    out_files = []
    merged_count = 0
    for i in range(0, len(segments), n_merge):
        chunk = segments[i:i+n_merge]
        if not chunk:
            continue
        merged_df = pd.concat(chunk, ignore_index=True)

        out_filename = f"{audio_id}_merged_{merged_count}.csv"
        out_path = os.path.join(VISUAL_OUTPUT_DIR, out_filename)
        merged_df.to_csv(out_path, index=False)
        out_files.append(out_path)

        merged_count += 1

    return out_files

# === Example run ===
files_300_visual = extract_visual_segments(audio_id=300, n_merge=5)
print("Visual segments for 300:", files_300_visual)

# Split and Save Segmented Audio/Visual Files to Google Drive

In [ ]:
import os
import pandas as pd
import numpy as np
from typing import List

import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
from pydub import AudioSegment

#############################
# 0) Settings and path specification
#############################
TRANSCRIPT_BASE_DIR = "/content/drive/MyDrive/DAIC-WOZ/extracted"

OUTPUT_AUDIO_DIR = "/content/drive/MyDrive/DAIC-WOZ/processed_patient_segments"
os.makedirs(OUTPUT_AUDIO_DIR, exist_ok=True)

OUTPUT_VISUAL_DIR = "/content/drive/MyDrive/DAIC-WOZ/processed_visual_segments"
os.makedirs(OUTPUT_VISUAL_DIR, exist_ok=True)

# Path to combined_PHQ_sorted.csv
PHQ_CSV_PATH = "/content/combined_PHQ_sorted.csv"


#############################
# 1) Function to read TRANSCRIPT
#############################
def load_transcript_custom(path):
    """
    Reads DAIC-WOZ's {ID}_TRANSCRIPT.csv.
    The first line is assumed to be a header (e.g. 'start_time stop_time speaker value').
    From the second line onwards:
      - The first three tokens: start_time, stop_time, speaker
      - From the 4th token onwards: the utterance value
    """
    rows = []
    with open(path, 'r', encoding='utf-8', errors='replace') as f:
        # Skip the header line
        _ = next(f, None)
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(maxsplit=3)
            if len(parts) < 4:
                continue

            start_str, stop_str, spk, val = parts
            start_time = float(start_str)
            stop_time  = float(stop_str)

            rows.append({
                'start_time': start_time,
                'stop_time':  stop_time,
                'speaker':    spk,
                'value':      val
            })

    df = pd.DataFrame(rows, columns=['start_time','stop_time','speaker','value'])
    return df


#############################
# 2) Extract participant's audio segments
#############################
def extract_patient_segments(audio_id, n_merge=5):
    """
    1) Read {audio_id}_TRANSCRIPT.csv and get rows where speaker=='Participant'
    2) Use pydub to cut out the .wav file
    3) Concatenate every n_merge segments -> write the result to OUTPUT_AUDIO_DIR
    4) Return a list of output file paths
    """
    subfolder = f"{audio_id}_P"
    transcript_file = f"{audio_id}_TRANSCRIPT.csv"
    audio_file      = f"{audio_id}_AUDIO.wav"
    transcript_path = os.path.join(TRANSCRIPT_BASE_DIR, subfolder, transcript_file)
    audio_path      = os.path.join(TRANSCRIPT_BASE_DIR, subfolder, audio_file)

    df = load_transcript_custom(transcript_path)
    df_patient = df[df['speaker'] == 'Participant'].copy()

    audio_full = AudioSegment.from_wav(audio_path)

    segments = []
    for _, row in df_patient.iterrows():
        start_ms = int(row['start_time'] * 1000)
        end_ms   = int(row['stop_time']  * 1000)
        seg = audio_full[start_ms:end_ms]
        segments.append(seg)

    out_files = []
    merged_count = 0
    for i in range(0, len(segments), n_merge):
        chunk_segs = segments[i:i+n_merge]
        if not chunk_segs:
            continue
        merged = sum(chunk_segs)
        out_filename = f"{audio_id}_merged_{merged_count}.wav"
        out_path = os.path.join(OUTPUT_AUDIO_DIR, out_filename)
        merged.export(out_path, format="wav")
        out_files.append(out_path)
        merged_count += 1

    return out_files


#############################
# 3) Load CLNF (visual) files
#############################
def load_clnf_data(audio_id: int) -> pd.DataFrame:
    """
    Merge the CLNF files in extracted/{audio_id}_P/ by timestamp.
    Skip the HOG file.
    * If the files are actually whitespace-delimited, set delim_whitespace=True.
    """
    subfolder = f"{audio_id}_P"
    folder_path = os.path.join(TRANSCRIPT_BASE_DIR, subfolder)

    clnf_filenames = [
        f"{audio_id}_CLNF_pose.txt",
        f"{audio_id}_CLNF_hog.txt",  # skip hog
        f"{audio_id}_CLNF_gaze.txt",
        f"{audio_id}_CLNF_features3D.txt",
        f"{audio_id}_CLNF_features.txt",
        f"{audio_id}_CLNF_AUs.txt",
    ]

    df_merged = None
    for fn in clnf_filenames:
        fp = os.path.join(folder_path, fn)
        if not os.path.exists(fp):
            print(f"[WARN] Not found: {fp}, skip.")
            continue

        if "hog" in fn.lower():
            print(f"[INFO] Skipping HOG file: {fn}")
            continue

        try:
            with open(fp, "r", encoding="utf-8", errors="replace") as f:
                df_temp = pd.read_csv(f, sep=",", header=0)
        except UnicodeDecodeError:
            print(f"[ERROR] Could not decode {fn}. Skipping...")
            continue

        # Strip whitespace from column names
        df_temp.columns = [c.strip() for c in df_temp.columns]

        # If timestamp is missing, create it from frame (assuming fps=30)
        if "timestamp" not in df_temp.columns and "frame" in df_temp.columns:
            df_temp["timestamp"] = df_temp["frame"] / 30.0

        if df_merged is None:
            df_merged = df_temp
        else:
            df_merged.columns = [c.strip() for c in df_merged.columns]
            df_merged = pd.merge(
                df_merged, df_temp, on="timestamp", how="outer",
                suffixes=("", "_dup")
            )

    if df_merged is None:
        return pd.DataFrame()

    df_merged = df_merged.sort_values("timestamp").reset_index(drop=True)
    return df_merged


def extract_visual_segments(audio_id, n_merge=5):
    """
    1) Retrieve Participant segments via load_transcript_custom
    2) From the merged DataFrame in load_clnf_data, cut out rows whose timestamps lie within [start_time, stop_time]
    3) Concatenate every n_merge segments -> output CSV -> return a list of paths
    """
    subfolder = f"{audio_id}_P"
    transcript_file = f"{audio_id}_TRANSCRIPT.csv"
    transcript_path = os.path.join(TRANSCRIPT_BASE_DIR, subfolder, transcript_file)

    df_trans = load_transcript_custom(transcript_path)
    df_patient = df_trans[df_trans['speaker'] == 'Participant'].copy()

    df_clnf = load_clnf_data(audio_id)
    if df_clnf.empty:
        print(f"[WARN] No CLNF data for {audio_id}")
        return []

    segments = []
    for _, row in df_patient.iterrows():
        start_sec = row['start_time']
        stop_sec  = row['stop_time']
        mask = (df_clnf["timestamp"] >= start_sec) & (df_clnf["timestamp"] <= stop_sec)
        seg_df = df_clnf[mask].copy()
        segments.append(seg_df)

    out_files = []
    merged_count = 0
    for i in range(0, len(segments), n_merge):
        chunk = segments[i:i+n_merge]
        if not chunk:
            continue
        merged_df = pd.concat(chunk, ignore_index=True)
        out_filename = f"{audio_id}_merged_{merged_count}.csv"
        out_path = os.path.join(OUTPUT_VISUAL_DIR, out_filename)
        merged_df.to_csv(out_path, index=False)
        out_files.append(out_path)
        merged_count += 1

    return out_files


#############################
# 4) Collect segment extractions for all participants
#############################
def gather_all_segments(id_list, n_merge=5, label_dict=None):
    """
    For each ID in id_list:
      - Extract audio segments with extract_patient_segments
      - Extract visual segments with extract_visual_segments
      - Pair them up by the same index and assign label_dict[ID] as the label
    """
    audio_paths_all = []
    visual_paths_all = []
    labels_all = []

    for pid in id_list:
        audio_list = extract_patient_segments(pid, n_merge=n_merge)
        visual_list = extract_visual_segments(pid, n_merge=n_merge)

        # Label (for example, PHQ8_Binary)
        lbl = 0.0
        if label_dict and pid in label_dict:
            lbl = label_dict[pid]

        min_len = min(len(audio_list), len(visual_list))
        for i in range(min_len):
            audio_paths_all.append(audio_list[i])
            visual_paths_all.append(visual_list[i])
            labels_all.append(lbl)

    return audio_paths_all, visual_paths_all, labels_all

# Generate a Multimodal (Audio/Visual/Text) Dataset

<h3>Collect and Match Audio/Visual Segments by Participant ID, Then Save to CSV</h3>

In [ ]:
import os
import pandas as pd

# ----------------------------------------------------------------
# 1) Retrieve ID -> label (PHQ8_Binary) from combined_PHQ_sorted.csv
# ----------------------------------------------------------------
df_phq = pd.read_csv(PHQ_CSV_PATH, sep=",")  # Adjust the delimiter as appropriate for your data
# If it's tab-delimited, change to sep="\t"

# Assuming there are "Participant_ID" and "PHQ8_Binary" columns
# Convert labels to a dictionary, e.g., label_dict[300] = 0, label_dict[301] = 1, ...
label_dict = dict(zip(df_phq["Participant_ID"], df_phq["PHQ8_Binary"]))


# ----------------------------------------------------------------
# 2) From processed_* folders, match {ID}_merged_*.wav with {ID}_merged_*.csv
# ----------------------------------------------------------------
audio_paths_all = []
visual_paths_all = []
labels_all = []

# Iterate over the ID -> label dictionary (includes IDs from 300 to 492 and possibly others)
for pid, label in label_dict.items():
    # Look for files containing pid in their names
    # Example: "300_merged_0.wav", "300_merged_1.wav", etc.
    wav_files = [fn for fn in os.listdir(AUDIO_DIR)
                 if fn.startswith(f"{pid}_merged_") and fn.endswith(".wav")]

    csv_files = [fn for fn in os.listdir(VISUAL_DIR)
                 if fn.startswith(f"{pid}_merged_") and fn.endswith(".csv")]

    # Function to extract the merged index (e.g., 0, 1, 12) from the filename
    def get_merged_index(filename):
        # Example filename: "300_merged_12.wav" or "300_merged_12.csv"
        base, _ = os.path.splitext(filename)  # => "300_merged_12"
        # Split after "300_merged_"
        parts = base.split("_merged_")
        if len(parts) == 2:
            # parts[0] = "300", parts[1] = "12"
            return int(parts[1])  # convert "12" to 12
        else:
            return None

    # Sort the files by the index at the end of the filename
    wav_files_sorted = sorted(wav_files, key=get_merged_index)
    csv_files_sorted = sorted(csv_files, key=get_merged_index)

    # Match up audio and visual files for the minimum number of segments in common
    # (e.g., if there are 5 audio segments and 7 visual segments, only match 5)
    min_len = min(len(wav_files_sorted), len(csv_files_sorted))

    for i in range(min_len):
        # Audio file path
        audio_path = os.path.join(AUDIO_DIR, wav_files_sorted[i])
        # Visual CSV file path
        visual_path = os.path.join(VISUAL_DIR, csv_files_sorted[i])

        audio_paths_all.append(audio_path)
        visual_paths_all.append(visual_path)
        labels_all.append(label)


# ----------------------------------------------------------------
# 3) Save the final matches to CSV
# ----------------------------------------------------------------
df_out = pd.DataFrame({
    "audio_path": audio_paths_all,
    "visual_path": visual_paths_all,
    "label": labels_all
})

df_out.to_csv(OUTPUT_CSV, index=False)
print(f"[INFO] Saved {len(df_out)} rows to {OUTPUT_CSV}")

<h3>Merge Text Modality Predictions with Audio/Visual Dataset by Participant ID</h3>

In [ ]:
import os
import pandas as pd
import re

# ★ 1) Read dataset_info_all.csv
DATASET_INFO_CSV = "/content/drive/MyDrive/DAIC-WOZ/dataset_info_all.csv"
df_main = pd.read_csv(DATASET_INFO_CSV)

# The columns in df_main are assumed to be ['audio_path','visual_path','label'], for example:
# /content/drive/MyDrive/DAIC-WOZ/processed_patient_segments/301_merged_0.wav

# Define a function to extract the "Participant ID" from the audio_path
def extract_id_from_path(path):
    """
    Example:
      path = "/content/drive/.../processed_patient_segments/301_merged_0.wav"
      => participant_id = 301 (int)
    """
    # Get only the filename
    filename = os.path.basename(path)  # => "301_merged_0.wav"

    # Use a regular expression to extract the ID
    # Example pattern: [digits]+(?=_merged_)
    match = re.match(r"(\d+)_merged_\d+", filename)
    if match:
        return int(match.group(1))
    else:
        return None

# Add a "participant_id" column to df_main
df_main["participant_id"] = df_main["audio_path"].apply(extract_id_from_path)

# ★ 2) Read text-modality-result.csv
TEXT_MODALITY_CSV = "/content/text-modality-result.csv"
df_text = pd.read_csv(TEXT_MODALITY_CSV)

# df_text is assumed to have columns -> ['ID','o1pro-prediction']
# Rename the ID column to "participant_id" to facilitate merging
df_text = df_text.rename(columns={"ID": "participant_id"})

# ★ 3) Merge
df_merged = pd.merge(
    df_main,
    df_text,
    on="participant_id",  # Merge on participant_id
    how="left"            # Left join => rows from df_main remain; if an ID is not found, the new columns are NaN
)

# Now df_merged has the "o1pro-prediction" column (e.g., "depressed" or "not depressed")

# ★ 4) Save to CSV (audio_path, visual_path, label, participant_id, o1pro-prediction)
OUTPUT_CSV = "/content/drive/MyDrive/DAIC-WOZ/dataset_info_all_text.csv"
df_merged.to_csv(OUTPUT_CSV, index=False)
print(f"[INFO] Saved: {OUTPUT_CSV}")